### Noise

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times"]})

from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from sklearn.metrics import r2_score

In [ ]:
from wPCC_pipeline.pipelines.preprocess.nodes import filter, assemble_data, add_thrust
from wPCC_pipeline.pipelines.extended_kalman.nodes import create_extended_kalman
from wPCC_pipeline.pipelines.brix.nodes import initial_parameters
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter,extended_kalman_smoother
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import guess_covariance_matrixes, extended_kalman_filter, initial_state, extended_kalman_smoother
from wPCC_pipeline.pipelines.preprocess.nodes import load, filter
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler

In [ ]:
#ship = "kvlcc2_hsva"
ship = "wpcc"
#id = model_test_ids[ship][0]
#id = "22774"
id = "22771"
raw_data = catalog.load(f"{ship}.{ id }.data")
ship_data = catalog.load(f"{ship}.ship_data")
vmm_name = "vmm_martins_simple"
vmm = catalog.load(vmm_name)
ek = catalog.load(f"{ ship }.{ vmm_name }.ek")
model = catalog.load(f"{ ship }.{ vmm_name }.ek")

data = load(raw_data, replace_velocities=True)

cuttoffs = np.linspace(1.0,10.0, 10)
datas_lowpass = {}
for cuttoff in cuttoffs:
    datas_lowpass[cuttoff] = filter(raw_data, cutoff=cuttoff, order=1)

data_ek_smooth = catalog.load(f"{ ship }.updated.{ id }.data_ek_smooth")

In [ ]:
data_ek_smooth['U'] = data_ek_smooth['V'] = np.sqrt(data_ek_smooth['u']**2 + data_ek_smooth['v']**2)

In [ ]:
added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")

In [ ]:
columns = ['u','v','r','u1d','v1d','r1d','delta','thrust','x0','y0','psi']
data_with_force_smooth = predict_force(data=data_ek_smooth[columns], 
                                added_masses=added_masses, 
                                ship_parameters=ship_data, 
                                vmm=vmm)

#data_with_force_smooth.plot(y='fy')


In [ ]:

columns = ['u','v','r','u1d','v1d','r1d','delta','thrust','x0','y0','psi']
data_with_force = predict_force(data=data[columns], 
                                added_masses=added_masses, 
                                ship_parameters=ship_data, 
                                vmm=vmm)

data_with_force_smooth = predict_force(data=data_ek_smooth[columns], 
                                added_masses=added_masses, 
                                ship_parameters=ship_data, 
                                vmm=vmm)


models_lowpass = {}
datas_with_force_lowpass = {}
regressions_lowpass = {}
for cuttoff, data_lowpass in datas_lowpass.items():
    
    datas_with_force_lowpass[cuttoff] = predict_force(data=data_lowpass[columns], 
                                added_masses=added_masses, 
                                ship_parameters=ship_data, 
                                vmm=vmm)
    
    regressions_lowpass[cuttoff], parameters_lowpass = fit_motions(data=datas_with_force_lowpass[cuttoff], 
                                     added_masses=added_masses, 
                                     ship_data=ship_data, vmm=vmm, 
                                     exclude_parameters=exclude_parameters)

    models_lowpass[cuttoff] = create_model_from_motion_regression(regression=regressions_lowpass[cuttoff])


In [ ]:
regression, parameters = fit_motions(data=data_with_force, 
                                     added_masses=added_masses, 
                                     ship_data=ship_data, vmm=vmm, 
                                     exclude_parameters=exclude_parameters)

regression_smooth, parameters_smooth = fit_motions(data=data_with_force_smooth, 
                                     added_masses=added_masses, 
                                     ship_data=ship_data, vmm=vmm, 
                                     exclude_parameters=exclude_parameters)


In [ ]:
model = create_model_from_motion_regression(regression=regression)
model_smooth = create_model_from_motion_regression(regression=regression_smooth)


In [ ]:
i_max = 3000
#results = model.simulate(data.iloc[0:i_max])
#results_smooth = model_smooth.simulate(data.loc[0:i_max])

results = simulate_euler(data_ek_smooth.iloc[0:i_max], model=model, ek=ek).dropna()
results_smooth = simulate_euler(data_ek_smooth.iloc[0:i_max], model=model_smooth, ek=ek).dropna()

dataframes_lowpass = {}
for cuttoff, model_ in models_lowpass.items():
    dataframes_lowpass[cuttoff] = simulate_euler(data_ek_smooth.iloc[0:i_max], model=model_, ek=ek).dropna()

In [ ]:
dataframes = {
    'raw' : data.iloc[0:i_max],
    'simulation' : results,
    'simulation smooth' : results_smooth,
    
}
dataframes.update(dataframes_lowpass)
styles = {
    'raw' : {'style':'k-'},
    'simulation' : {'style':'--'},
    'simulation smooth' : {'style':'--'},
    
}

for cuttoff in models_lowpass.keys():
    styles[cuttoff] = {'style':'--'}
    

ax = track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B'],N=5, styles=styles, flip=True);
ax.set_ylim(-1,5)
ax.set_xlim(-5,40)



In [ ]:
#dofs = ['u','v','r']
dofs = ['x0','y0','psi']
df_r2 = pd.DataFrame(columns=dofs, index=dataframes.keys())

for dof in df_r2:
    for key, prediction in dataframes.items():
        try:
            df_r2.loc[key,dof] = r2_score(y_true = data[dof].iloc[0:i_max], y_pred=prediction[dof])
        except:
            continue

            
df_r2['mean'] = df_r2.mean(axis=1)
df_r2.sort_values(by=['mean'], inplace=True, ascending=False)
#df_r2.index = df_r2.index.astype(str)


In [ ]:
index_lowpass = [i for i in df_r2.index if not isinstance(i,str)]

fig,ax=plt.subplots()
df_r2.loc[index_lowpass].sort_index().plot(y='mean', ax=ax, label='Lowpass')
ax.plot([cuttoffs[0],cuttoffs[-1]], [df_r2.loc['simulation smooth','mean'],df_r2.loc['simulation smooth','mean']], label='EKF + smoother')
ax.set_ylim(0.95,1)
ax.set_ylabel('Simulation accuracy ($r^2$)')
ax.set_xlabel('Low-pass cut off frequency [Hz]')
ax.grid(True)
ax.legend()


In [ ]:
keys=['u']

dataframes_ = {

    'raw EK' : data_ek_smooth,
    'raw lowpass' : datas_lowpass[df_r2.drop(index=['raw', 'simulation smooth']).index[0]],
    'simulation smooth' : results_smooth,
    
}
dataframes_.update(dataframes_lowpass)
dataframes_.pop(1)
styles = {
    'raw lowpass' : {'style':'m-'},
    'raw EK' : {'style':'k:'},
    
    'simulation' : {'style':'--'},
    'simulation smooth' : {'style':'--'},
    
}

fig = plot(dataframes=dataframes_, keys=keys, styles=styles,ncols=1, time_window=[0,30], zero_origo=False);

#y_scale = 2.5
#for key, ax in zip(keys,fig.axes):
#    ax.set_ylim(y_scale*data_ek_smooth[key].min(), y_scale*data_ek_smooth[key].max())

In [ ]:
columns = ['U','u','v','r','u1d','v1d','r1d','delta','thrust','x0','y0','psi']

data_prediction = regression.predict(data=data_ek_smooth[columns])
data_prediction_smooth = regression_smooth.predict(data=data_ek_smooth[columns])

In [ ]:
fig,ax=plt.subplots()
data_with_force_smooth.plot(y='fy', ax=ax)
data_prediction_smooth.plot(y='fy', ax=ax)

regression_smooth.predict_parameter_contributions_Y().plot(ax=ax)
regression_smooth.predict_parameter_contributions_Y().sum(axis=1).plot(ax=ax, label='sum', style='--')
ax.legend()
